# GOAL : Scrape Drama Reviews
FROM: https://mydramalist.com/

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time


In [2]:
driver_place = 'C:/Users/Hanna Kim/Documents/데잇걸즈 4기/공공데이터 공모전/chromedriver'
driver = webdriver.Chrome(driver_place)
driver.get('https://mydramalist.com/profile/dragynfaerie/reviews?page=1')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [30]:
def get_frame(soup):
    title = []
    reviewer = []
    # create list for each column
    story = []
    acting = []
    music = []
    rewatch = []
    tire = soup.select("a.text-primary")
    score = soup.select("span.p-l-md.pull-right")
    for i in range(int(len(tire)/3)):
        title.append(tire[3*i].text)
        reviewer.append(tire[3*i+1].text)
        story.append(score[4*i].text) 
        acting.append(score[4*i+1].text)
        music.append(score[4*i+2].text)
        rewatch.append(score[4*i+3].text)
    overall = [*map(lambda x:x.text,soup.select("span.score"))]
    
    return pd.DataFrame({"title":title, "reviewer":reviewer, "overall":overall,"story":story,"acting":acting, "music":music, "rewatch":rewatch})

In [53]:
#고정된 driver page열어주기
driver = webdriver.Chrome(driver_place)

In [48]:
def create_url(userid):
    url = f'https://mydramalist.com/profile/{userid}/reviews?page='
    return url

def get_pg(n):
    driver.get(url +f'{n}')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [54]:
url = create_url('dragynfaerie')

In [45]:
'''
soup = get_pg(1)
a =1
while soup.select_one("li.page-item.nb.active").text == str(a):
    print("same")
    if soup.select_one("li.page-item.nb.active").text != str(a):
        print("not same")
        break
    a +=1
    soup = get_pg(a)
'''

'\nsoup = get_pg(1)\na =1\nwhile soup.select_one("li.page-item.nb.active").text == str(a):\n    print("same")\n    if soup.select_one("li.page-item.nb.active").text != str(a):\n        print("not same")\n        break\n    a +=1\n    soup = get_pg(a)\n'

In [42]:
soup = get_pg(1)
a =1
df = get_frame(soup)
while soup.select_one("li.page-item.nb.active").text == str(a):
    if soup.select_one("li.page-item.nb.active").text != str(a): 
        break
    a +=1
    soup = get_pg(a)
    df2 = get_frame(soup)
    df = pd.concat([df,df2])

same
same
same
same


In [56]:
def create_url(userid):
    url = f'https://mydramalist.com/profile/{userid}/reviews?page='
    return url

def get_pg(n):
    driver.get(url +f'{n}')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def get_one_user_frame(userid):
    url = create_url(userid)
    soup = get_pg(1)
    a =1
    df = get_frame(soup)
    while soup.select_one("li.page-item.nb.active").text == str(a):
        if soup.select_one("li.page-item.nb.active").text != str(a): 
            break
        a +=1
        soup = get_pg(a)
        df2 = get_frame(soup)
        df = pd.concat([df,df2])
    return df

In [57]:
get_one_user_frame("dragynfaerie")

,title,reviewer,overall,story,acting,music,rewatch
0,Goblin,dragynfaerie,7.5,6.5,9.0,8.0,5.0
1,The Sound of Your Heart,dragynfaerie,10,10,10,9.0,10
2,Sword and Flower,dragynfaerie,8.0,9.0,10,9.0,5.0
3,Who Are You,dragynfaerie,7.0,6.0,9.0,7.0,6.0
4,Once Again,dragynfaerie,9.5,8.0,10,9.0,9.5
...,...,...,...,...,...,...,...
6,Fullmetal Alchemist,dragynfaerie,9.0,9.0,10,8.0,8.0
7,The Funeral,dragynfaerie,10,8.5,9.0,7.0,9.5
8,Detective Dee and the Mystery of the Phantom F...,dragynfaerie,8.0,8.0,9.0,8.0,10
9,"Human, Space, Time and Human",dragynfaerie,1.0,4.0,2.0,5.0,1.0
